In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import validators
import phonenumbers
import re
# from requests.exceptions import SSLError
from zenrows import ZenRowsClient

In [ ]:
# Find PhoneNo

def website_code(domain):
    emails = []
    new_domain = 'https://' + domain
    client = ZenRowsClient(API)
    response = client.get(new_domain)
#     response = requests.get(new_domain, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    data = soup.text
    booddy = []
    body1 = soup.find('body')
    booddy.append(body1)
    new_data = [data]
    links = soup.find_all('a')
    
    for link in links:
        text = link.text.lower()
        href = link.get('href')

        # Look for the keywords in the text of the link
        if 'contact' in text or 'contact us' in text:
            # Validate the link
            if validators.url(str(href)):
                new_response = client.get(href)
                new_soup = BeautifulSoup(new_response.content, 'html.parser')
                new_data.append(new_soup.text)
                body2 = new_soup.find('body')
                booddy.append(body2)
                
#     email_pattern = r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+"
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b'
#     email_pattern = r"""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"""
    found_emails = re.findall(email_pattern, str(booddy), re.I)
    emails = found_emails

    class PhoneNumberExtractor:
        def extract(self, text):
            phone_numbers = []
            for match in phonenumbers.PhoneNumberMatcher(text, None):
                phone_number = match.number
                international_format = phonenumbers.format_number(phone_number, phonenumbers.PhoneNumberFormat.INTERNATIONAL)
                country_code = phonenumbers.region_code_for_number(phone_number)
                phone_numbers.append({'number': international_format, 'country_code': country_code})
            return phone_numbers

    extractor = PhoneNumberExtractor()
    all_phone_numbers = []
    
    for i_data in new_data:
        extracted_phone_numbers = extractor.extract(i_data)
        all_phone_numbers.extend(extracted_phone_numbers)
    
    if not all_phone_numbers:
        phone_number_pattern = re.compile(r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}')
        phone_numbers = phone_number_pattern.findall(data)
        
        if phone_numbers:
            return phone_numbers,emails
        else:
            return "None",emails

    # Convert the list of dictionaries to a set of tuples to remove duplicates
    all_phone_numbers_set = set(tuple(phone_number.items()) for phone_number in all_phone_numbers)
    
    # Convert the set of tuples back to a list of dictionaries
    all_phone_numbers_no_duplicates = [dict(phone_number) for phone_number in all_phone_numbers_set]

    # Return the list with no duplicates
    return all_phone_numbers_no_duplicates, emails
#     return all_phone_numbers

def process_numbers(domain):
    results, emails = website_code(domain)
    no1, no2, c_code1, c_code2 = '', '', '', ''
    
    if isinstance(results, list):
        if len(results) > 0:
            if isinstance(results[0], dict):
                no1 = results[0].get('number', '')
                c_code1 = results[0].get('country_code', '')
                
                if len(results) > 1:
                    no2 = results[1].get('number', '')
                    c_code2 = results[1].get('country_code', '')
            else:
                no1 = results[0]
                c_code1 = ''
    
    return no1, no2, c_code1, c_code2,emails

In [ ]:
# api1 = '4cf789df8f4651ff43acab1f140cc02763c74cfb'
# API2 = '0ccd1e7b95e12b45adec3e26f4413ed97db5aed8'
# API3 = '8fa012728b42aca349cdbf1c4a8b073f8708c35e'
# api1 = '44885dae07cb9c92084a86a73a462e171cab6f76'
# API2 = '6b2af6128310918de808dc19db8536bd094f69b7'
API2 = '20cd8c840b0f0057a1e2056d43c9edaae6c0cb3f'
new = [API2]
name_company = []
email_company_1 = []
all_email_list = []
phone_no1 = []
country_code1 = []
phone_no2 = []
country_code2 = []
Company = pd.read_excel('Company.xlsx')

In [ ]:
count = 0
for i, clean_domain in zip(Company['Company Name'], Company['Website']):
    API = new[count % len(new)]
    count += 1
    domain = clean_domain
    no1, no2, c_code1, c_code2,emailslist = process_numbers(domain)
    name_company.append(i)
    phone_no1.append(no1)
    phone_no2.append(no2)
    country_code1.append(c_code1)
    country_code2.append(c_code2)
    all_email_list.append(emailslist)
    if emailslist:
        email_company_1.append(emailslist[0])
        em = emailslist[0]
    else:
        email_company_1.append('')
        em = ''
    print(len(name_company), len(email_company_1), em, no1, '   ', API[0])

In [22]:
# Assuming all lists have the same length
data = {
    'Name': name_company,
    'Email 1': email_company_1,
    'Phone 1': phone_no1,
    'Country Code 1': country_code1,
    'Phone 2': phone_no2,
    'Country Code 2': country_code2,
    'Email List': all_email_list
}

df = pd.DataFrame(data)

In [23]:
df

,Name,Email 1,Phone 1,Country Code 1,Phone 2,Country Code 2,Email List
0,Alchem,contact@alcheminternational.com,+91 11 2680 9882,IN,+41 91 604 69 21,CH,"[contact@alcheminternational.com, contact@alch..."
1,Nutreemax,info@nutreemax.com,+92 322 2610770,PK,,,"[info@nutreemax.com, info@nutreemax.com, info@..."
2,Junaid,,,,,,[]
3,Texchem Wave,texchemwave@gmail.com,+880 1723-179343,BD,+880 1794-534474,BD,"[texchemwave@gmail.com, texchemwave@gmail.com,..."
4,Shandong Tongli Chemical Co.,,,,,,[]
...,...,...,...,...,...,...,...
940,Roquette FrÃ¨res,,,,,,[]
941,Agrana Group,,,,,,[]
942,SÃ¼dzucker AG,,,,,,[]
943,Tereos Group,contactgdpr@tereos.com,,,,,"[contactgdpr@tereos.com, contactgdpr@tereos.com]"


In [24]:
df.to_csv('all_con.csv')